In [ ]:
import sys 
sys.path.insert(0,"../")

import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import jax.numpy as jnp
from jax import value_and_grad,grad,jit
import jax.lax as lax
import jax

from src.utils import batch_dataset

# Load data

In [ ]:
dataset = jnp.load("./dataset/dataset.npy")
masks = jnp.load("./dataset/masks.npy")
metadata = pd.read_csv("./dataset/metadata.csv",index_col=0)

In [ ]:
batched_dataset, batched_masks = batch_dataset(dataset,10,masks)

In [ ]:
from src.barycenter import batch_varifold_barycenter_registration
from src.kernel import TSGaussGaussKernel,TSGaussDotKernel,VFTSGaussKernel
import optax

In [ ]:
batched_dataset = batched_dataset[:2]
batched_masks = batched_masks[:2]
Kv = VFTSGaussKernel(1,0.1,30,1,1)
Kl = TSGaussGaussKernel(2,2,2,2)
#Kl = TSGaussDotKernel(10,1,1,1,2)
#ps ,qb, qb_mask= batch_varifold_barycenter_registration(batched_dataset,batched_masks,Kv,Kl,niter=200,optimizer=optax.adam(learning_rate=0.1),gamma_loss=0.00)

In [ ]:
def get_raw_signal(sig,mask): 
    idx = np.where(mask==True)[0]
    return sig[idx]

In [ ]:
for sig,mask in zip(dataset[:60],masks[:60]): 
    plt.plot(*get_raw_signal(sig,mask)[:,::2].T,color="black",alpha=0.5)

sig =  get_raw_signal(qb,qb_mask)
plt.plot(*sig[:,::2].T,color="red")

In [ ]:
from src.masked.lddmm import Flowing,Shooting
from src.plotting import plot2Dfigure

In [ ]:
shoot = Shooting(Kv)
p,q = shoot(ps[0,1],qb,qb_mask)

In [ ]:
plt.plot(*get_raw_signal(batched_dataset[0,1],batched_masks[0,1])[:,::2].T,label="target")
plt.plot(*get_raw_signal(q,qb_mask)[:,::2].T,label="project")
plt.plot(*get_raw_signal(qb,qb_mask)[:,::2].T,label="barycenter")
plt.legend()

In [ ]:
from src.statistic import MomentaPCA

In [ ]:
tps = ps.reshape(-1,ps.shape[-2],ps.shape[-1])

In [ ]:
metadata.PathologyGroup[:60].values

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

In [ ]:
label = OrdinalEncoder().fit_transform(metadata.PathologyGroup[:60].values.reshape(-1,1)).squeeze()

In [ ]:
mpca = MomentaPCA(3,masked=True)
mpca.fit(Kv,tps,qb,qb_mask)
plt.scatter(*mpca.p_score_[:,[0,2]].T,c=label)

In [ ]:
import json 

In [ ]:
json.dumps(ps.tolist())